# Library import

In [1]:
import pandas as pd
import numpy as np
import itertools
import random

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import scipy.cluster.hierarchy as shc
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings(action='ignore')

# Function Definations

In [2]:
#Tukey's method defination
def tukeys_method(df, variable):
    #Takes two parameters: dataframe & variable of interest as string
    q1 = df[variable].quantile(0.25)
    q3 = df[variable].quantile(0.75)
    iqr = q3-q1
    inner_fence = 1.5*iqr
    outer_fence = 3*iqr
    
    #inner fence lower and upper end
    inner_fence_le = q1-inner_fence
    inner_fence_ue = q3+inner_fence
    
    #outer fence lower and upper end
    outer_fence_le = q1-outer_fence
    outer_fence_ue = q3+outer_fence
    
    outliers_prob = []
    outliers_poss = []
    for index, x in enumerate(df[variable]):
        if x <= outer_fence_le or x >= outer_fence_ue:
            outliers_prob.append(index)
    for index, x in enumerate(df[variable]):
        if x <= inner_fence_le or x >= inner_fence_ue:
            outliers_poss.append(index)
    return outliers_prob, outliers_poss

# EDD function
def descriptive_statistics_summary(df):
    df_desc_statistics = df.describe().transpose()
    variable_type = df.dtypes
    variable_count = df.count()
    miss_val_count = df.isnull().sum()
    miss_val_percent = 100 * df.isnull().sum()/len(df)
    unique_count = df.nunique()
    
    median = df.median().astype('object')
    quan_0_1 = df.quantile(0.01).transpose()
    quan_1 = df.quantile(0.1).transpose()
    quan_99 = df.quantile(.99).transpose()
    
    columns = df.keys()
    
    index = 0
    
    for i, col in enumerate(columns):
        col_dtype = df[col].dtypes
        
        if col_dtype == 'object':
            unique_counts = df[col].value_counts(dropna = 0)
            median.at[col] = str(unique_counts.index[0]) + ':' + str(unique_counts[0])
            quan_0_1.at[col] = str(unique_counts.index[1]) + ':' + str(unique_counts[1]) if len(unique_counts) >= 2 else "0"
            quan_1.at[col] = str(unique_counts.index[2]) + ':' + str(unique_counts[2]) if len(unique_counts) >= 3 else "0"
            quan_99.at[col] = str(unique_counts.index[3]) + ':' + str(unique_counts[3]) if len(unique_counts) >= 4 else "0"
            df_desc_statistics.at[col, 'mean'] = str(unique_counts.index[4]) + ':' + str(unique_counts[4]) if len(unique_counts) >= 5 else "0"

        
    miss_val_table = pd.concat(
            [
                variable_type,
                variable_count,
                miss_val_count,
                miss_val_percent,
                unique_count,
                median,
                quan_0_1,
                quan_1,
                quan_99,
                df_desc_statistics['mean'],
                df_desc_statistics['std'],
                df_desc_statistics['min'],
                df_desc_statistics['25%'],
                df_desc_statistics['50%'],
                df_desc_statistics['75%'],
                df_desc_statistics['max']
            ], axis=1).rename(
    columns = {
                0 : 'Variable Type',
                1 : 'Variable Count',
                2 : 'Missing Value Count',
                3 : '% Total Missing Values',
                4 : 'Unique Count',
                5 : 'Median',
                6 : '1%'
    })
    
    return miss_val_table

# pre-post export
def pp_export(pre_data,post_data,filename):
    with pd.ExcelWriter(f'{output_path}/{filename}.xlsx') as writer:
        pre_data.to_excel(writer, sheet_name='Pre_treatment',index=False)
        post_data.to_excel(writer, sheet_name='Post_treatment',index=False)
        descriptive_statistics_summary(pre_data).to_excel(writer, sheet_name='EDD_Pre_treatment',index=True)
        descriptive_statistics_summary(post_data).to_excel(writer, sheet_name='EDD_Post_treatment',index=True)

        
def get_binary_col(df,value_dict):
    for col in value_dict:
        val_1 = list(value_dict[f'{col}'][0].keys())[0]
        val_2 = list(value_dict[f'{col}'][0].keys())[1]
        
        val_1_list = value_dict[f'{col}'][0][val_1]
        val_2_list = value_dict[f'{col}'][0][val_2]
        
        df[f'{col}'] = df[f'{col}'].apply(lambda x : val_1 if x in val_1_list else (val_2 if x in val_2_list else None))
    
        return df

# Data import

In [3]:
input_path = './Input_data'
output_path = './Output_data'
data_orig = pd.read_csv(f"{input_path}/Masked_Mercury Financial Sample.csv")

FileNotFoundError: [Errno 2] No such file or directory: './Input_data/Masked_Mercury Financial Sample.csv'

In [ ]:
data = data_orig.copy()
data.head(10)

In [ ]:
data.shape

# Data preparation

### Data merge

In [ ]:
# Internal + Bureau datasets
# data = pd.merge()

### Deleting insignificant records and vars

In [ ]:
data_pre = data.copy()

In [ ]:
# unrequired columns
redundant_cols = ['Sr. no'
                ,'CE_Individual_HoH_Title_Code'
                ,'CE_Buyer_Behavior_Cluster_Code_Filler'
                ,'CE_Delivery_Point'
                ,'CE_House_Fraction'
                ,'CE_Match_Level'
                ,'CE_Match_Score'
                ,'CE_Route_Number'
                 ]
data.drop(columns=redundant_cols, inplace=True)

In [ ]:
# threshold for null values across columns
record_na_thresh = int(data.shape[1] * 0.5)
drop_record_index = list(data[data.isna().sum(axis=1) > record_na_thresh].index)
data.drop(index=drop_record_index,axis=0, inplace=True)

In [ ]:
data.shape

In [ ]:
pp_export(data_pre,data,'1.Insig_record_drop')

# Data Processing

### Exploratory Data Analysis (EDA)

### *Insights* : 
- Data contains integer and categorical value columns
- There are no missing values
- Columns 'Experience' and 'Current_Job_Years' are corelated
- No sign for outliers

In [ ]:
data.describe()

In [ ]:
data.info()

### Heatmap/Corelation matrix

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))
sns.heatmap(data.corr(),cmap="YlGnBu", annot=True, ax=ax)
plt.show()

### Observations distribution

In [ ]:
numeric_cols = list(data.select_dtypes(exclude=['object']).columns)
for col in data.columns:
    if col in numeric_cols:
        sns.displot(data=data, x=col, kind='kde', fill=True, palette=sns.color_palette('bright'), height=5, aspect=2.5)
        plt.show()

In [ ]:
for col in data.columns:
    if col in numeric_cols:
        sns.boxplot(data=data, y=col, palette=sns.color_palette('muted'))
        plt.show()

### Data transformation, encoding, formatting

In [ ]:
data.nunique()

#### Encoding
- Binary encoding on 'Married/Single' and 'Car_Ownership' columns
- One hot encoding on House ownership column
- Frequency encoding on 'Profession' and 'STATE' columns

In [ ]:
data_pre = data.copy()

In [ ]:
value_matching_dict = {'EDUCATION': [{
                               'Bachelor': ['BTECH'],
                               'Master': ['MTECH','MBA']
                                    }]
                      }

data = get_binary_col(data,value_matching_dict)

In [ ]:
data.nunique()

In [ ]:
# Binary encoding
col_uniq_count = dict(data.nunique())

binary_enc_cols = [key for key in col_uniq_count if col_uniq_count[key] == 2 and key not in numeric_cols]
# binary_enc_cols = ['Married/Single','Car_Ownership','MISS_CAT','EDUCATION']


for col in binary_enc_cols:
    uni_col_vals = list(data[f'{col}'].unique())
    uni_col_vals = [val for val in uni_col_vals if str(val).lower() not in ['','nan','na','null']]
    data[f'{col}'] = data[f'{col}'].apply(lambda x: 0 if x == uni_col_vals[0] else (1 if x == uni_col_vals[1] else None))
#     data = data.drop(columns=[col])

data.head()

In [ ]:
binary_enc_cols

In [ ]:
pp_export(data_pre,data,'2.Binary_encoding')

In [ ]:
data_pre = data.copy()

In [ ]:
# one hot encoding

# selecting columns for encoding
oneHot_enc_cols = [key for key in col_uniq_count if col_uniq_count[key] in ([3,4,5]) and key not in numeric_cols]
# oneHot_enc_cols = ['House_Ownership']

for col in oneHot_enc_cols:
    # creating dummy varibles df
    dummy_df = pd.get_dummies(data[f'{col}'], prefix=f'{col}')
    
    # dummy variables merge into data
    dummy_df.drop(columns=dummy_df.columns[0], inplace=True)
    data = pd.concat([data,dummy_df],axis='columns')
    
    # dropping base column
    data = data.drop(columns=[col])
    
data.head()

In [ ]:
oneHot_enc_cols

In [ ]:
pp_export(data_pre,data,'3.One_hot_encoding')

In [ ]:
data_pre = data.copy()

In [ ]:
# Frequency encoding

# selecting columns for encoding
freq_enc_cols = [key for key in col_uniq_count if col_uniq_count[key] >5 and key not in numeric_cols]
# freq_enc_cols = ['Profession','CITY','STATE']

for col in freq_enc_cols:
    # calculating frequency
    col_freq = data.groupby(f'{col}').size()/len(data)
    
    # mapping values
    data.loc[:, f"{col}_enc"] = round(data[f'{col}'].map(col_freq),5)
    
    # dropping base column`
    data = data.drop(columns=[col])
    
data.head()

In [ ]:
freq_enc_cols

In [ ]:
pp_export(data_pre,data,'4.Freq_encoding')

In [ ]:
base_data = data.copy()

### Dropping corelated columns

In [ ]:
data_pre = data.copy()

In [ ]:
# heatmap over encoded data
fig, ax = plt.subplots(figsize=(15, 8))
sns.heatmap(data.corr(),cmap="YlGnBu", annot=True, ax=ax)
plt.show()

In [ ]:
# Set the corelation threshold
corr_thresh = 0.60

cor_matrix = data.corr().abs()
upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(bool))
to_drop_cols = [column for column in upper_tri.columns if any(upper_tri[column] > corr_thresh)]

data.drop(columns=to_drop_cols,inplace=True)

In [ ]:
data.shape

In [ ]:
# Sanity check after dropping corelated columns
fig, ax = plt.subplots(figsize=(15, 8))
sns.heatmap(data.corr(),cmap="YlGnBu", annot=True, ax=ax)
plt.show()

In [ ]:
pp_export(data_pre,data,'5.Corr_var_drop')

### Dropping insignificant vars

In [ ]:
data_pre = data.copy()

In [ ]:
# Dropping insignificant varibales

# dropping null values columns based on missing values threshold
missing_threshold = 0.4

non_misisng_count = int((1-missing_threshold) * data.shape[0] + 1)
data.dropna(thresh=non_misisng_count, axis=1, inplace=True)

In [ ]:
pp_export(data_pre,data,'6.Insig_var_drop')

In [ ]:
data.shape

### Missing value treatment

In [ ]:
data_pre = data.copy()

In [ ]:
# Filling binary columns with same ratio
for col in [x for x in binary_enc_cols if x in data.columns]:
    total_null = data[col].isna().sum()
    total_filled = (data.shape[0]) - total_null
    
    filled_ratio_val1 = round((data[col].value_counts()[0])/total_filled,1)
    filled_ratio_val2 = 1-filled_ratio_val1
    
    fillna_val1_count = int(filled_ratio_val1 * total_null)
    fillna_val2_count = int(total_null-fillna_val1_count)
    
    null_index = list(data[col].index[data[col].apply(np.isnan)])
    
    val1_index = random.sample(null_index, fillna_val1_count)
    val2_index = [val for val in null_index if val not in val1_index]
    
    data.loc[val1_index,col] = data[col].unique()[0]
    data.loc[val2_index,col] = data[col].unique()[1]

In [ ]:
data.info()

In [ ]:
data.CE_Individual_HoH_Has_Misc_Credit_Card.unique()

In [ ]:
data.head(10)

In [ ]:
# REMOVE THIS STEP
data = data.select_dtypes(exclude='object')

In [ ]:
# filling remaining null values with mean/median/mode
treatment_cols = [col for col in data.loc[:,data.isna().sum() > 0].columns]

### Uncomment below section
for col in treatment_cols:
#     fill_value = data[f'{col}'].mean()
    fill_value = data[f'{col}'].median()
#     fill_value = data[f'{col}'].mode()
    data.fillna(value=fill_value, inplace=True)

In [ ]:
pp_export(data_pre,data,'7.Missing_val_treatment')

### Outlier Treatment

In [ ]:
data_pre = data.copy()

In [ ]:
data.reset_index(inplace=True)

In [ ]:
outlier_cols = [x for x in data.columns if x not in binary_enc_cols]
outlier_cols = [x for x in outlier_cols if x not in oneHot_enc_cols]
outlier_cols

In [ ]:
# getting indices for outliers in numeric columns
outliers_indices_prob = []
outliers_indices_poss = []

# for col in [x for x in numeric_cols if x in data.columns and not in (binary_enc_cols and oneHot_enc_cols)]:
for col in outlier_cols:
    outliers_prob, outliers_poss = tukeys_method(data,col)
    outliers_indices_prob.append(outliers_prob)
    outliers_indices_poss.append(outliers_poss)
    
# dropping outliers indices
drop_poss = 1   #0-> Probable Outliers | 1-> Possible Outliers

if drop_poss:
    combined_indices = itertools.chain.from_iterable(outliers_indices_poss)
    drop_list = list(set(list(combined_indices)))
else:
    combined_indices = itertools.chain.from_iterable(outliers_indices_prob)
    drop_list = list(set(list(combined_indices)))
    
data.drop(index=drop_list, axis=0, inplace=True)

In [ ]:
[x for x in outlier_cols if x not in freq_enc_cols]

In [ ]:
data.shape

In [ ]:
for col in data.columns:
    if col in numeric_cols:
        sns.boxplot(data=data, y=col, palette=sns.color_palette('muted'))
        plt.show()

In [ ]:
for col in data.columns:
    if col in [x for x in numeric_cols if x in data.columns]:
        sns.boxplot(data=data, y=col, palette=sns.color_palette('muted'))
        plt.show()

In [ ]:
pp_export(data_pre,data,'8.Outlier_records_drop')

# Pre-Clustering

## Feature Scaling
- Since data doesn't follow normal curve distribution Normalization is preferred 

In [ ]:
data_pre = data.copy()

In [ ]:
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

print(f'Scaled data shape : {data_scaled.shape}')

In [ ]:
scaled_final_data = pd.DataFrame(data_scaled, columns=data.columns)
scaled_final_data.head()

In [ ]:
pp_export(data_pre,scaled_final_data,'9.Feature_scaling')

## PCA

In [ ]:
data_pre = scaled_final_data.copy()

In [ ]:
# PCA with Variance(information) cut-off as 95%

# pca = PCA(0.95)
# principal_components = pca.fit_transform(scaled_final_data)
# principal_df = pd.DataFrame(data = principal_components)
# principal_df

In [ ]:
# PCA with Variance(information) cut-off as 80%

# pca2 = PCA(0.80)
# principal_components2 = pca2.fit_transform(scaled_final_data)
# principal_df2 = pd.DataFrame(data = principal_components2)
# principal_df2

In [ ]:
# PCA with Variance(information) cut-off as 70%

pca3 = PCA(0.70)
principal_components3 = pca3.fit_transform(scaled_final_data)
principal_df3 = pd.DataFrame(data = principal_components3 , columns=list(f'Principal_component_{i + 1}' for i in range(len(principal_components3[0]))))
principal_df3.head()

In [ ]:
# Components with respective variance accountibility with variance(information) cutoff as 95%
# pca.explained_variance_ratio_

In [ ]:
# Components with respective variance accountibility with variance(information) cutoff as 80%
# pca2.explained_variance_ratio_

In [ ]:
# Components with respective variance accountibility with variance(information) cutoff as 70%
pca3.explained_variance_ratio_

In [ ]:
pp_export(data_pre,principal_df3,'10.PCA')

### Dendrograms with 3 variance cut-offs

In [ ]:
# plt.figure(figsize=(20,10))
# plt.title("Dendrograms")
# dend = shc.dendrogram(shc.linkage(principal_df, method='ward'),show_leaf_counts=True)
# plt.show()

In [ ]:
# plt.figure(figsize=(20,10))
# plt.title("Dendrograms")
# dend = shc.dendrogram(shc.linkage(principal_df2, method='ward'),show_leaf_counts=True)
# plt.show()

In [ ]:
# plt.figure(figsize=(20,10))
# plt.title("Dendrograms")
# dend = shc.dendrogram(shc.linkage(principal_df3, method='ward'),show_leaf_counts=True)
# plt.show()

In [ ]:
cluster_num = [2,3,4]

In [ ]:
for x in cluster_num:
    cluster = AgglomerativeClustering(n_clusters=x, affinity='euclidean', linkage='ward')  
    cluster_label3 = cluster.fit_predict(principal_df3)
    print(silhouette_score(principal_df3, cluster_label3))

### Clustering with n=4 based on the above dendrogram

In [ ]:
# Getting cluster labels
cluster = AgglomerativeClustering(n_clusters=4, affinity='euclidean', linkage='ward')  
# cluster_label = cluster.fit_predict(principal_df)

# cluster_label2 = cluster.fit_predict(principal_df2)

cluster_label3 = cluster.fit_predict(principal_df3)

In [ ]:
# dropping the delted records from base data
base_data.drop(index=drop_list, axis=0, inplace=True)

In [ ]:
# Creating cluster label rows in original data

# data_orig['Cluster'] = cluster_label
# data_orig['Cluster2'] = cluster_label2
base_data['Cluster'] = cluster_label3
base_data.head(15)

### Comparing Cluster label counts with different PCA outputs

In [ ]:
# data_orig.Cluster.value_counts()

In [ ]:
# data_orig.Cluster2.value_counts()

In [ ]:
base_data.Cluster.value_counts()

In [ ]:
# Outputting the clustered data | Uncomment the below line

base_data.to_csv(f"{output_path}/Clustered_data.csv",index=False)